In [1]:
import os
import shutil 
import csv
import numpy as np
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import StandardScaler
from utils import *

Using TensorFlow backend.


In [2]:
band_name = 'v8'  
window_size = 6 

In [25]:
path = 'sources/Ini_sources/'
#print(os.getcwd())
data_1 = make_sequences_y1(band_name, path, band_name)
print(len(data_1))
print(data_1)

10
[[3.6325600000000122, 3.630423999999948, 3.626299999999901, 3.6202769999999873, 3.613411000000042, 3.605570999999941, 3.598645000000033, 3.5925459999999703, 3.5861169999999447, 3.5819499999998925, 3.5790140000000292, 3.5772160000000213, 3.5766069999999672, 3.577682999999979, 3.579494000000068, 3.5832960000000185, 3.5874300000000403, 3.5926919999999427, 3.5994069999999283], [3.5875939999999673, 3.5860830000000306, 3.58442500000001, 3.5824029999999993, 3.580515000000105, 3.5785640000000285, 3.5768000000000484, 3.575134000000048, 3.574037999999973, 3.5739620000000514, 3.573853999999983, 3.574555000000032, 3.578021999999919, 3.578895999999986, 3.5815470000000005, 3.586221000000023, 3.591946000000007, 3.5999000000001615], [3.5837850000000344, 3.5758050000000594, 3.5654899999999543, 3.5505940000000464, 3.5326430000000073, 3.5115349999999808, 3.4878740000000334, 3.4620459999999866, 3.434436000000005, 3.4057840000000397, 3.3765769999999975, 3.3477499999999054, 3.319975000000113, 3.293734999

In [4]:
''' Writing out initial sequences
file_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')
for line in data_1:
    file_ini.write(str(line) +'\n')
    
file_ini.close()
'''

" Writing out initial sequences\nfile_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')\nfor line in data_1:\n    file_ini.write(str(line) +'\n')\n    \nfile_ini.close()\n"

In [5]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().rounding = ROUND_HALF_UP
l = len(data_1)
h = int(round(Decimal(l)/2,0))
i = int(round(Decimal(h)/2,0))
indexes = [i, l-(h-i)]
print(indexes)

[3, 8]


In [6]:
train = data_1[:indexes[0]] + data_1[indexes[1]:]
test  = data_1[indexes[0]:indexes[1]]
print(len(train))

5


In [7]:
'''
data_test_1 = get_x(test, window_size=window_size)

file_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
#filewriter.writerow([1,2,3,4,5,6,'Y'])
filewriter.writerow([1,2,3,4,5,6,'Y1','Y2','Y3','Y4','Y5','Y6'])
for i in range(data_test_1.shape[0]):
    filewriter.writerow(data_test_1[i,:])
file_csv.close()
'''

"\ndata_test_1 = get_x(test, window_size=window_size)\n\nfile_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')\nfilewriter = csv.writer(file_csv, delimiter=',',\n                            quotechar='|', quoting=csv.QUOTE_MINIMAL)\n#filewriter.writerow([1,2,3,4,5,6,'Y'])\nfilewriter.writerow([1,2,3,4,5,6,'Y1','Y2','Y3','Y4','Y5','Y6'])\nfor i in range(data_test_1.shape[0]):\n    filewriter.writerow(data_test_1[i,:])\nfile_csv.close()\n"

# Creating additional samples for train set

In [75]:
from statistics import mean 

def check_data_uniform_chaning(data, out=False):
    data_to_continue = []
    for line in data:
        flag = True
        temp = line[1] - line[0]
        if temp > 0:
            for i in range(1, len(line[:-1])):
                if line[i+1] < line[i]:
                    flag = False
                    break
        else:
            for i in range(1,len(line[:-1])):
                if line[i+1] > line[i]:
                    flag = False
                    break
        if flag is False:
            if out is True: print(line)
            new = []
            #m = mean(line)
            #for i in line:
            #    new.append(i-m)
            #print(new)
        else:
            data_to_continue.append(line)
    return data_to_continue


In [51]:
def get_labels_y1(data):
    data_new = []
    for i in range(len(data)):
        data_new.append(data[i] + [1,1,1,1,1,1])
    return data_new

In [73]:
def negative_sequences(data):
    data_new = []
    for el in data:
        seq = []
        for cell in el:
            seq.append(-cell)
        data_new.append(seq)

    return data_new

In [86]:
def reverse_sequences(data, out=False):
    data_new = []
    for line in data:
        line_new = line[::-1]
        if out is True:
            print('ini:\t', line)
            print('reversed:\t', line_new)
        data_new.append(line_new)
    return data_new

In [95]:
def calculate_deltas(line):
    deltas = [0]
    for i in range(len(line)-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

def create_shift_in_deltas(data, shift, out=False):
    data_add = []

    for line in data:
        deltas = calculate_deltas(line)
        min_delta = min(deltas[1:])
        if abs(shift) > abs(min_delta):
            continue
        new = [line[0]]
        ini = line[0]
        for i in range(1, len(line)):
            ini = ini - deltas[i] - shift
            new.append(ini)
        if out is True:
            print('ini:\t', line)
            print('ini deltas:\t', deltas)
            print('new:\t', new)
        data_add.append(new)
    
    return data_add

In [114]:
file_csv = open('sources/train_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y1','Y2','Y3','Y4','Y5','Y6'])


31

In [115]:
data = get_x(train, window_size=window_size).tolist()
print('ini:\t\t', len(data))

data_add_negative = negative_sequences(data)
print('negative:\t', len(data_add_negative))

data += data_add_negative
print('total:\t\t', len(data))

data_out = get_labels_y1(data)

data_out = np.array(data_out)
for i in range(data_out.shape[0]):
    filewriter.writerow(data_out[i,:])


ini:		 61
negative:	 61
total:		 122


In [116]:
data = check_data_uniform_chaning(data, out=False)
print('only uniform changing in sequence:\t', len(data))   

only uniform changing in sequence:	 106


In [117]:
data_add_reversed = reverse_sequences(data, out=False)
print('ini:\t\t', len(data))
print('reversed:\t', len(data_add_reversed))

data += data_add_reversed
print('total:\t\t', len(data))

ini:		 106
reversed:	 106
total:		 212


In [118]:
print('ini:\t\t', len(data))
shifts = [i for i in np.arange(-0.015, 0.015, 0.0005) if abs(i) > 0.00001]
data_add_res = []
for shift in shifts:
    data_add = create_shift_in_deltas(data, shift)
    data_add_res += data_add
print('shifted:\t', len(data_add_res))

data += data_add_res
print('total:\t\t', len(data))

ini:		 212
shifted:	 9300
total:		 9512


In [119]:
data_out = get_labels_y1(data)

data_out = np.array(data_out)
for i in range(data_out.shape[0]):
    filewriter.writerow(data_out[i,:])
    

In [120]:
file_csv.close()

In [ ]:
'''Muliplicate deltas by 10s
data_add = []
for line in data:
    deltas = create_deltas(line, y_label=True)
    #print(line)
    #print('d ', deltas)
    if all(delta <= 0.1 for delta in deltas) is True:
        cell_max = m
        ax(map(abs,deltas))
        cell_min = min(map(abs,deltas[1:]))
        while (cell_max < 0.05):
            deltas = [d*10 for d in deltas]
            cell_max *= 10
            new = []
            ini = line[0]
            for i in range(len(deltas)):
                ini = ini - deltas[i]
                new.append(ini)
            new.append(1.)
            if abs(line[0] - 3.5738) <= 0.0001:
                print(line)
                print(create_deltas(line, y_label=True))
                print('1', new)
    
            #print('n ',new)
            data_add.append(new)
            
            if cell_min >= 0.002:
                deltas2 = [d/2 for d in deltas]
                new = []
                #print('1', deltas)
                #print('2',deltas2)
                ini = line[0]
                for i in range(len(deltas2)):
                    ini = ini - deltas2[i]
                    new.append(ini)
                new.append(1.)
                if abs(line[0] - 3.5738) <= 0.0001:
                    print('2',new)
                #print('n ',new)
                data_add.append(new)
            
print(len(data_add))
'''